In [1]:
_ = !pip3 install -r requirements.txt

In [2]:
%load_ext tensorboard

In [3]:
%tensorboard --logdir logs --port=6006 --bind_all

In [4]:
from utils import plot_learning_curves, build_data

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [6]:
import sklearn as skl
from sklearn.metrics import f1_score, accuracy_score

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *

In [8]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
train = build_data('train')
val = build_data('val')

In [10]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor = 'val_expression_output_loss', 
        patience = 3,
        restore_best_weights = True
    ),
    keras.callbacks.ModelCheckpoint(
        'models/efficient.h5',
        monitor="val_expression_output_loss",
        save_weights_only=False,
        save_best_only=True,
        verbose=1,
        mode="auto",
        save_freq="epoch",
        period = 1
    ),
    keras.callbacks.TensorBoard(log_dir="logs/", update_freq=3500)
]

In [11]:
EPOCHS = 50
BATCH_SIZE = 32

In [12]:
input_shape = (224, 224, 3)

In [13]:
generator = ImageDataGenerator(
    rotation_range=10, # rotation
    width_shift_range=0.2, # horizontal shift
    height_shift_range=0.2, # vertical shift
    zoom_range=0.2, # zoom
    horizontal_flip=True, # horizontal flip
    validation_split = .15
)
train_gen = generator.flow_from_dataframe(
    dataframe = train,
    x_col = 'image_path',
    y_col = ['valence', 'arousal', 'expression'],
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'multi_output',
    subset = 'training',
    batch_size = BATCH_SIZE,
    preprocessing_function = keras.applications.efficientnet.preprocess_input,
    shuffle = True
)

val_gen = generator.flow_from_dataframe(
    dataframe = train,
    x_col = 'image_path',
    y_col = ['valence', 'arousal', 'expression'],
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'multi_output',
    subset = 'validation',
    batch_size = BATCH_SIZE,
    preprocessing_function = keras.applications.efficientnet.preprocess_input,
    shuffle = True
)

Found 244504 validated image filenames.
Found 43147 validated image filenames.


In [14]:
test_gen = ImageDataGenerator().flow_from_dataframe(
    dataframe = val,
    x_col = 'image_path',
    y_col = ['valence', 'arousal', 'expression'],
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'multi_output',
    batch_size = BATCH_SIZE,
    preprocessing_function = keras.applications.efficientnet.preprocess_input,
    shuffle = True
)

Found 3999 validated image filenames.


In [15]:
def make_efficient():
    inp = Input(input_shape)
    
    base = keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_tensor=inp,
        pooling=None,
    )
    
    base.trainable = False
    
    flatten = Flatten()(base.output)
    
    dense1 = Dense(256, kernel_regularizer = 'l2')(flatten)
    dense1 = BatchNormalization()(dense1)
    dense1 = ReLU()(dense1)
    dense1 = Dropout(.5)(dense1)

    dense2 = Dense(128, kernel_regularizer = 'l2')(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = ReLU()(dense2)
    dense2 = Dropout(.5)(dense2)
    
    val_out = Dense(1, activation=None, name = 'valence_output')(dense2)
    aro_out = Dense(1, activation=None, name = 'arousal_output')(dense2)
    exp_out = Dense(8, activation="softmax", name = 'expression_output')(dense2)
    
    model = keras.Model(inputs = inp, outputs = [val_out, aro_out, exp_out])
    
    return model, base

In [16]:
model, base = make_efficient()

16711680/16705208 [==============================] - 1s 0us/step


In [17]:
model.compile(loss=["mse", "mse", "sparse_categorical_crossentropy"],
                  metrics = {
                      'valence_output':[tf.keras.metrics.RootMeanSquaredError()],
                      'arousal_output':[tf.keras.metrics.RootMeanSquaredError()],
                      'expression_output':['accuracy']},
                  optimizer='adam')

In [18]:
history = model.fit(
        x = train_gen,
        epochs = 2,
        validation_data = val_gen,
        verbose = 1
    )

Epoch 1/2
7641/7641 [==============================] - 1708s 223ms/step - loss: 4.0602 - valence_output_loss: 0.2841 - arousal_output_loss: 0.1069 - expression_output_loss: 1.4386 - valence_output_root_mean_squared_error: 0.5315 - arousal_output_root_mean_squared_error: 0.3268 - expression_output_accuracy: 0.5039 - val_loss: 2.8854 - val_valence_output_loss: 0.2072 - val_arousal_output_loss: 0.1060 - val_expression_output_loss: 1.3223 - val_valence_output_root_mean_squared_error: 0.4552 - val_arousal_output_root_mean_squared_error: 0.3255 - val_expression_output_accuracy: 0.5392
Epoch 2/2
7641/7641 [==============================] - 1702s 223ms/step - loss: 2.9001 - valence_output_loss: 0.2239 - arousal_output_loss: 0.1051 - expression_output_loss: 1.3475 - valence_output_root_mean_squared_error: 0.4732 - arousal_output_root_mean_squared_error: 0.3242 - expression_output_accuracy: 0.5250 - val_loss: 2.7280 - val_valence_output_loss: 0.2142 - val_arousal_output_loss: 0.1060 - val_expres

In [19]:
base.trainable = True

In [20]:
model.compile(loss=["mse", "mse", "sparse_categorical_crossentropy"],
                  metrics = {
                      'valence_output':[tf.keras.metrics.RootMeanSquaredError()],
                      'arousal_output':[tf.keras.metrics.RootMeanSquaredError()],
                      'expression_output':['accuracy']},
                  optimizer=keras.optimizers.Adam(1e-5))

In [21]:
history_finetune = model.fit(
        x = train_gen,
        epochs = EPOCHS,
        validation_data = val_gen,
        callbacks = callbacks,
        verbose = 1
    )

Epoch 1/50
7641/7641 [==============================] - 1744s 228ms/step - loss: 2.0616 - valence_output_loss: 0.1819 - arousal_output_loss: 0.1053 - expression_output_loss: 1.1823 - valence_output_root_mean_squared_error: 0.4259 - arousal_output_root_mean_squared_error: 0.3246 - expression_output_accuracy: 0.5967 - val_loss: 1.3015 - val_valence_output_loss: 0.1362 - val_arousal_output_loss: 0.1060 - val_expression_output_loss: 0.9188 - val_valence_output_root_mean_squared_error: 0.3691 - val_arousal_output_root_mean_squared_error: 0.3255 - val_expression_output_accuracy: 0.6878

Epoch 00001: val_expression_output_loss improved from inf to 0.91882, saving model to models/efficient.h5
Epoch 2/50
7641/7641 [==============================] - 1738s 227ms/step - loss: 1.3161 - valence_output_loss: 0.1422 - arousal_output_loss: 0.1052 - expression_output_loss: 0.9482 - valence_output_root_mean_squared_error: 0.3771 - arousal_output_root_mean_squared_error: 0.3243 - expression_output_accurac

7641/7641 [==============================] - 1738s 227ms/step - loss: 0.9466 - valence_output_loss: 0.1232 - arousal_output_loss: 0.1050 - expression_output_loss: 0.6592 - valence_output_root_mean_squared_error: 0.3511 - arousal_output_root_mean_squared_error: 0.3241 - expression_output_accuracy: 0.7744 - val_loss: 0.9518 - val_valence_output_loss: 0.1242 - val_arousal_output_loss: 0.1060 - val_expression_output_loss: 0.6625 - val_valence_output_root_mean_squared_error: 0.3524 - val_arousal_output_root_mean_squared_error: 0.3255 - val_expression_output_accuracy: 0.7655

Epoch 00023: val_expression_output_loss improved from 0.66272 to 0.66254, saving model to models/efficient.h5
Epoch 24/50
7641/7641 [==============================] - 1736s 227ms/step - loss: 0.9483 - valence_output_loss: 0.1243 - arousal_output_loss: 0.1052 - expression_output_loss: 0.6596 - valence_output_root_mean_squared_error: 0.3526 - arousal_output_root_mean_squared_error: 0.3243 - expression_output_accuracy: 0.7

In [22]:
history_finetune

In [26]:
model.evaluate(val_gen)

1349/1349 [==============================] - 257s 190ms/step - loss: 0.9504 - valence_output_loss: 0.1239 - arousal_output_loss: 0.1060 - expression_output_loss: 0.6620 - valence_output_root_mean_squared_error: 0.3520 - arousal_output_root_mean_squared_error: 0.3255 - expression_output_accuracy: 0.7690


[0.9503548741340637,
 0.12387217581272125,
 0.10596499592065811,
 0.6619651317596436,
 0.35195478796958923,
 0.32552266120910645,
 0.769045352935791]

In [25]:
model.save('models/efficientB0_Affect.h5')